# Trabajo Práctico 4 - Big Data UNT
## Clasificación y regularización de Desocupación usando la EPH

### Parte 1: Análisis de la base de hogares y tipo de ocupación

#### Ejercicio 2

In [4]:
import pandas as pd

In [5]:
#Cargamos la base de datos de los hogares de la EPH para el primer trimestre de 2004 y 2024

#basehogar2004=pd.read_stata("/Users/vfuentesmortensen/Library/CloudStorage/OneDrive-Personal/Documentos/Maestría UNT/Big Data/GitHub/TP0-Grupo2/TP4/Hogar_t104.dta", convert_categoricals=False)
#basehogar2024=pd.read_excel("/Users/vfuentesmortensen/Library/CloudStorage/OneDrive-Personal/Documentos/Maestría UNT/Big Data/GitHub/TP0-Grupo2/TP4/usu_hogar_T124.xlsx")
basehogar2004=pd.read_stata("C:/Users/Guadalupe Gorostiaga/Desktop/MAESTRÍA UNT/5. Big Data/TPs/TP4/Hogar_t104.dta", convert_categoricals=False)
basehogar2024=pd.read_excel("C:/Users/Guadalupe Gorostiaga/Desktop/MAESTRÍA UNT/5. Big Data/TPs/TP4/usu_hogar_T124.xlsx")

#Unificamos los nombres de las columnas de ambos data frames dejando todo en minúsculas
basehogar2004.columns=basehogar2004.columns.str.lower()
basehogar2024.columns=basehogar2024.columns.str.lower()

#Eliminamos los aglomerados que no sean Gran SMT y Tafí Viejo
basehogar2004=basehogar2004[basehogar2004['aglomerado']==29]
basehogar2024=basehogar2024[basehogar2024['aglomerado']==29]

In [7]:
#Cargamos la base de datos individuales de la EPH para el primer trimestre de 2004 y 2024

#baseind2004=pd.read_stata("/Users/vfuentesmortensen/Library/CloudStorage/OneDrive-Personal/Documentos/Maestría UNT/Big Data/GitHub/TP0-Grupo2/TP3/usu_individual_T104.dta", convert_categoricals=False)
#baseind2024=pd.read_excel("/Users/vfuentesmortensen/Library/CloudStorage/OneDrive-Personal/Documentos/Maestría UNT/Big Data/GitHub/TP0-Grupo2/TP3/usu_individual_T124.xlsx")
baseind2004=pd.read_stata("C:/Users/Guadalupe Gorostiaga/Desktop/MAESTRÍA UNT/5. Big Data/TPs/TP4/usu_individual_T104.dta", convert_categoricals=False)
baseind2024=pd.read_excel("C:/Users/Guadalupe Gorostiaga/Desktop/MAESTRÍA UNT/5. Big Data/TPs/TP4/usu_individual_T124.xlsx")

#Unificamos los nombres de las columnas de ambos data frame dejando todo en minúsculas
baseind2004.columns=baseind2004.columns.str.lower()
baseind2024.columns=baseind2024.columns.str.lower()

#Eliminamos los aglomerados que no sean Gran SMT y Tafí Viejo
baseind2004=baseind2004[baseind2004['aglomerado']==29]
baseind2024=baseind2024[baseind2024['aglomerado']==29]

In [8]:
#Observamos la estructura de las bases antes y después del filtrado de las observaciones

filas, columnas = basehogar2004.shape
print(f"Estructura de la base de hogares de 2004 es: {filas} filas y {columnas} columnas.")

filas, columnas = basehogar2024.shape
print(f"Estructura de la base de hogares de 2024 es: {filas} filas y {columnas} columnas.")

filas, columnas = baseind2004.shape
print(f"Estructura de la base de individuos de 2004 es: {filas} filas y {columnas} columnas.")

filas, columnas = baseind2024.shape
print(f"Estructura de la base de individuos de 2024 es: {filas} filas y {columnas} columnas.")

Estructura de la base de hogares de 2004 es: 661 filas y 88 columnas.
Estructura de la base de hogares de 2024 es: 639 filas y 88 columnas.
Estructura de la base de individuos de 2004 es: 2547 filas y 176 columnas.
Estructura de la base de individuos de 2024 es: 2109 filas y 177 columnas.


In [9]:
#Unimos las bases hogar con las bases individuales para cada año
base2004=pd.merge(basehogar2004, baseind2004, on=['codusu','nro_hogar'], how='left', suffixes=('', '_dup'))
base2024=pd.merge(basehogar2024, baseind2024, on=['codusu', 'nro_hogar'], how='left', suffixes=('', '_dup'))  

# Eliminamos las columnas duplicadas con el sufijo '_dup'
columnas_duplicadas = [col for col in base2004.columns if col.endswith('_dup')]
base2004.drop(columns=columnas_duplicadas, inplace=True)
base2024.drop(columns=columnas_duplicadas, inplace=True)

#Ahora unimos las bases en una sola
base = pd.concat([base2004, base2024], ignore_index=True)

filas, columnas = base2004.shape
print(f"Estructura de la base de 2004 es: {filas} filas y {columnas} columnas.")
filas, columnas = base2024.shape
print(f"Estructura de la base de 2024 es: {filas} filas y {columnas} columnas.")
filas, columnas = base.shape
print(f"Estructura de la base completa: {filas} filas y {columnas} columnas.")

Estructura de la base de 2004 es: 2547 filas y 242 columnas.
Estructura de la base de 2024 es: 2109 filas y 243 columnas.
Estructura de la base completa: 4656 filas y 248 columnas.


In [10]:
#Observamos y ordenamos la base
base = base[['codusu', 'nro_hogar', 'componente'] + [col for col in base.columns if col not in ['codusu', 'nro_hogar', 'componente']]]
base = base.sort_values(by=['codusu', 'nro_hogar', 'componente'])
base

,codusu,nro_hogar,componente,realizada,ano4,trimestre,region,mas_500,aglomerado,pondera,...,pj1_1,pj2_1,pj3_1,idimpp,pondih,ch05,imputa,pondiio,pondii,pondih_dup
0,125463,1.0,1.0,1.0,2004.0,1.0,40.0,S,29.0,259.0,...,0.0,0.0,0.0,00000,NaN,NaN,NaN,NaN,NaN,NaN
1,125463,1.0,2.0,1.0,2004.0,1.0,40.0,S,29.0,259.0,...,0.0,0.0,0.0,00000,NaN,NaN,NaN,NaN,NaN,NaN
2,125463,1.0,3.0,1.0,2004.0,1.0,40.0,S,29.0,259.0,...,0.0,0.0,0.0,00000,NaN,NaN,NaN,NaN,NaN,NaN
3,125463,1.0,4.0,1.0,2004.0,1.0,40.0,S,29.0,259.0,...,0.0,0.0,0.0,00000,NaN,NaN,NaN,NaN,NaN,NaN
4,125463,1.0,5.0,1.0,2004.0,1.0,40.0,S,29.0,259.0,...,0.0,0.0,0.0,00000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913,TQSMNOSXWHMMKQCDEHPJB00856905,1.0,4.0,1.0,2024.0,1.0,40.0,S,29.0,357.0,...,NaN,NaN,NaN,NaN,0.0,1999-12-18 00:00:00,NaN,0.0,0.0,0.0
3857,TQSMNOURSHMOKMCDEHPJB00859671,1.0,1.0,1.0,2024.0,1.0,40.0,S,29.0,572.0,...,NaN,NaN,NaN,NaN,567.0,1991-08-16 00:00:00,NaN,572.0,565.0,567.0
3856,TQSMNOURSHMOKMCDEHPJB00859671,1.0,2.0,1.0,2024.0,1.0,40.0,S,29.0,572.0,...,NaN,NaN,NaN,NaN,567.0,2022-03-13 00:00:00,NaN,572.0,572.0,567.0
3433,TQTMNOSTXHKOKNCDEHPJB00858768,1.0,1.0,1.0,2024.0,1.0,40.0,S,29.0,362.0,...,NaN,NaN,NaN,NaN,405.0,1959-09-23 00:00:00,NaN,362.0,358.0,405.0


In [11]:
#Queremos ver que la imputación haya sido correcta

# Agrupamos y tomamos el primer valor de 'itf'
valores = base.groupby(['codusu', 'nro_hogar'])['itf'].first()

# Verificamos si hay más de un valor por grupo
if (base.groupby(['codusu', 'nro_hogar'])['itf'].nunique() == 1).all():
    print("Todos los valores de 'itf' son constantes para cada combinación de 'codusu' y 'nro_hogar'.")
else:
    print("Hay combinaciones de 'codusu' y 'nro_hogar' con valores diferentes de 'itf'.")


Todos los valores de 'itf' son constantes para cada combinación de 'codusu' y 'nro_hogar'.


#### Ejercicio 3

In [13]:
# Visualizamos una descripción rápida de los datos para un subconjunto de variables de interés

#Rename de variables de interés
base.rename(columns={'iv1': 'tipo_vivienda', 'iv6': 'tenencia_agua', 'iv8': 'tenencia_bano', 'ch04': 'sexo', 'ch06': 'edad', 'ch07': 'estado_civil', 'ch08': 'cobertura_medica', 'ch10': 'asistencia_educ'}, inplace=True)

base[['tipo_vivienda', 'tenencia_agua', 'tenencia_bano', 'itf', 'ipcf', 'sexo', 'edad', 'estado_civil', 'cobertura_medica', 'asistencia_educ', 'nivel_ed', 'estado', 'cat_ocup', 'cat_inac']].describe()

,tipo_vivienda,tenencia_agua,tenencia_bano,itf,ipcf,sexo,edad,estado_civil,cobertura_medica,asistencia_educ,nivel_ed,estado,cat_ocup,cat_inac
count,4656.000000,4656.000000,4656.000000,4.656000e+03,4.656000e+03,4656.000000,4656.000000,4656.000000,4656.000000,4656.000000,4656.000000,4656.000000,4656.000000,4656.000000
mean,1.236254,1.135524,1.019974,2.252260e+05,6.050565e+04,1.522122,31.994416,3.666022,2.747208,1.679338,3.450387,2.342139,1.162371,1.976804
std,0.449399,0.356461,0.141454,3.884766e+05,1.172145e+05,0.499564,21.692509,1.585221,2.357130,0.609670,1.884674,1.153891,1.393806,2.045849
min,1.000000,1.000000,0.000000,0.000000e+00,0.000000e+00,1.000000,-1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,1.000000,4.050000e+02,8.666667e+01,1.000000,14.000000,2.000000,1.000000,1.000000,2.000000,1.000000,0.000000,0.000000
50%,1.000000,1.000000,1.000000,1.170000e+03,2.666667e+02,2.000000,28.000000,5.000000,2.000000,2.000000,3.000000,3.000000,0.000000,1.000000
75%,1.000000,1.000000,1.000000,3.850000e+05,9.250000e+04,2.000000,48.000000,5.000000,4.000000,2.000000,5.000000,3.000000,3.000000,3.000000
max,5.000000,3.000000,2.000000,3.650000e+06,2.250000e+06,2.000000,97.000000,5.000000,23.000000,3.000000,7.000000,4.000000,4.000000,7.000000


In [14]:
#Tenemos personas con edades negativas, las eliminamos
base=base[base['edad']>=0] 

#Limpiamos outliers en las variables de ingreso total familiar e ingreso per capita familiar

#Tomamos el criterio de 2 desviaciones estándar de la media para filtrar los outliers. 
for col in ['itf', 'ipcf']:
     mean_values = base[col].mean()
     std_dev_values = base[col].std()

     lower_bound = mean_values - 2 * std_dev_values
     upper_bound = mean_values + 2 * std_dev_values

# Filtramos el DataFrame manteniendo solo los valores dentro de los límites
base = base[(base[col] >= lower_bound) & (base[col] <= upper_bound)]
base

,codusu,nro_hogar,componente,realizada,ano4,trimestre,region,mas_500,aglomerado,pondera,...,pj1_1,pj2_1,pj3_1,idimpp,pondih,ch05,imputa,pondiio,pondii,pondih_dup
0,125463,1.0,1.0,1.0,2004.0,1.0,40.0,S,29.0,259.0,...,0.0,0.0,0.0,00000,NaN,NaN,NaN,NaN,NaN,NaN
1,125463,1.0,2.0,1.0,2004.0,1.0,40.0,S,29.0,259.0,...,0.0,0.0,0.0,00000,NaN,NaN,NaN,NaN,NaN,NaN
2,125463,1.0,3.0,1.0,2004.0,1.0,40.0,S,29.0,259.0,...,0.0,0.0,0.0,00000,NaN,NaN,NaN,NaN,NaN,NaN
4,125463,1.0,5.0,1.0,2004.0,1.0,40.0,S,29.0,259.0,...,0.0,0.0,0.0,00000,NaN,NaN,NaN,NaN,NaN,NaN
5,125571,1.0,2.0,1.0,2004.0,1.0,40.0,S,29.0,296.0,...,0.0,0.0,0.0,00000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913,TQSMNOSXWHMMKQCDEHPJB00856905,1.0,4.0,1.0,2024.0,1.0,40.0,S,29.0,357.0,...,NaN,NaN,NaN,NaN,0.0,1999-12-18 00:00:00,NaN,0.0,0.0,0.0
3857,TQSMNOURSHMOKMCDEHPJB00859671,1.0,1.0,1.0,2024.0,1.0,40.0,S,29.0,572.0,...,NaN,NaN,NaN,NaN,567.0,1991-08-16 00:00:00,NaN,572.0,565.0,567.0
3856,TQSMNOURSHMOKMCDEHPJB00859671,1.0,2.0,1.0,2024.0,1.0,40.0,S,29.0,572.0,...,NaN,NaN,NaN,NaN,567.0,2022-03-13 00:00:00,NaN,572.0,572.0,567.0
3433,TQTMNOSTXHKOKNCDEHPJB00858768,1.0,1.0,1.0,2024.0,1.0,40.0,S,29.0,362.0,...,NaN,NaN,NaN,NaN,405.0,1959-09-23 00:00:00,NaN,362.0,358.0,405.0


In [15]:
# Revisamos el número de missing values luego de las correcciones de nuestros datos
subset=['tipo_vivienda', 'tenencia_agua', 'tenencia_bano', 'itf', 'ipcf', 'sexo', 'edad', 'estado_civil', 'cobertura_medica', 'asistencia_educ', 'nivel_ed', 'estado', 'cat_ocup', 'cat_inac']
nulos = base[subset].isnull().sum()
print(nulos)

tipo_vivienda       0
tenencia_agua       0
tenencia_bano       0
itf                 0
ipcf                0
sexo                0
edad                0
estado_civil        0
cobertura_medica    0
asistencia_educ     0
nivel_ed            0
estado              0
cat_ocup            0
cat_inac            0
dtype: int64


#### Ejercicio 4

Computamos cuatro variables que pueden ser relevantes para predecir desocupación y no están en la base.

In [28]:
#Uno: Proporción de personas ocupadas en el hogar

# Contamos la cantidad de personas ocupadas por hogar (numerador)
ocupados_por_hogar = base[base['estado'] == 1].groupby(['codusu', 'nro_hogar'])['estado'].count().reset_index()
ocupados_por_hogar.rename(columns={'estado': 'ocupados_por_hogar'}, inplace=True)

# Contamos el total de miembros del hogar (denominador)
total_miembros_por_hogar = base.groupby(['codusu', 'nro_hogar'])['estado'].count().reset_index()
total_miembros_por_hogar.rename(columns={'estado': 'total_miembros_por_hogar'}, inplace=True)

# Unimos ambos dataframes
proporciones = pd.merge(total_miembros_por_hogar, ocupados_por_hogar, on=['codusu', 'nro_hogar'], how='left')
print(proporciones) #ocupados_por_hogar genera missing values cuando no hay personas ocupadas en el hogar

# Reemplazamps los NaN con 0 en 'ocupados_por_hogar'
proporciones['ocupados_por_hogar'] = proporciones['ocupados_por_hogar'].fillna(0)

# Calculamos la proporción
proporciones['proporcion_ocupados_hogar'] = proporciones['ocupados_por_hogar'] / proporciones['total_miembros_por_hogar']

# Agregamos esta nueva variable al dataframe base
base = pd.merge(base, proporciones[['codusu', 'nro_hogar', 'proporcion_ocupados_hogar']], on=['codusu', 'nro_hogar'], how='left')

# Verificamos los resultados
print(base['proporcion_ocupados_hogar'])

                             codusu  nro_hogar  total_miembros_por_hogar  \
0                          125463          1.0                         4   
1                          125571          1.0                         4   
2                          125591          1.0                         3   
3                          126052          1.0                         2   
4                          126339          1.0                         1   
...                             ...        ...                       ...   
1211  TQSMNOSTXHKOKNCDEHPJB00857737        1.0                         4   
1212  TQSMNOSVQHJOMMCDEHPJB00795313        1.0                         5   
1213  TQSMNOSXWHMMKQCDEHPJB00856905        1.0                         4   
1214  TQSMNOURSHMOKMCDEHPJB00859671        1.0                         2   
1215  TQTMNOSTXHKOKNCDEHPJB00858768        1.0                         2   

      ocupados_por_hogar  
0                    1.0  
1                    2.0  
2     

In [40]:
print("Cantidad de missing values en proporcion_ocupados_hogar: ", base['proporcion_ocupados_hogar'].isna().sum())
print("Cantidad de missing values en ocupados_por_hogar: ", proporciones['ocupados_por_hogar'].isnull().sum())
print("Cantidad de missing values en total_miembros_por_hogar: ", proporciones['total_miembros_por_hogar'].isnull().sum())

Cantidad de missing values en proporcion_ocupados_hogar:  0
Cantidad de missing values en ocupados_por_hogar:  0
Cantidad de missing values en total_miembros_por_hogar:  0


In [42]:
#Dos: Nivel educativo alcanzado

print(base['nivel_ed'].unique())

def clasificar_educacion(nivel):
    if nivel in [1, 7, 9] :
        return 'bajo'
    elif nivel in [2, 3]:
        return 'medio'
    elif nivel in [4, 5, 6]:
        return 'alto'
base['categoria_educacion'] = base['nivel_ed'].apply(clasificar_educacion)

print(base['categoria_educacion'])
print("Cantidad de missing values en categoria_educacion: ", base['proporcion_ocupados_hogar'].isna().sum())

[4. 5. 1. 2. 7. 6. 3.]
0        alto
1        alto
2        bajo
3        bajo
4        bajo
        ...  
4417     alto
4418    medio
4419     bajo
4420    medio
4421     alto
Name: categoria_educacion, Length: 4422, dtype: object
Cantidad de missing values en categoria_educacion:  0


In [44]:
# Removemos las observaciones que no declaran ningún ambiente de uso exclusivo en el hogar
print(base['ii1'].describe())
base = base[base['ii1'] != 0]
print("base shape:", base.shape)  # (filas, columnas)

count    4422.000000
mean        3.206920
std         1.356885
min         1.000000
25%         2.000000
50%         3.000000
75%         4.000000
max         9.000000
Name: ii1, dtype: float64
base shape: (4422, 251)


In [46]:
#Tres: Hacinamiento 

# Contamos la cantidad de ambientes/habitaciones que tiene el hogar
ambientes_hogar = base.groupby(['codusu', 'nro_hogar'])['ii1'].mean().reset_index()
ambientes_hogar.rename(columns={'ii1': 'ambientes_hogar'}, inplace=True)

# Contamos el total de miembros del hogar
total_miembros_por_hogar = base.groupby(['codusu', 'nro_hogar'])['ii1'].count().reset_index()
total_miembros_por_hogar.rename(columns={'ii1': 'total_miembros_por_hogar'}, inplace=True)

# Unimos ambos dataframes
densidad_habitacion = pd.merge(total_miembros_por_hogar, ambientes_hogar, on=['codusu', 'nro_hogar'], how='left')

# Eliminamos los hogares con ambientes cero
#print(densidad_habitacion[densidad_habitacion['ambientes_hogar'] == 0])
#densidad_habitacion = densidad_habitacion[densidad_habitacion['ambientes_hogar'] != 0]

# Paso 4: Calcular la proporción
densidad_habitacion['densidad_habitacion'] = densidad_habitacion['total_miembros_por_hogar'] / densidad_habitacion['ambientes_hogar']

# Paso 5: Unir las proporciones al DataFrame original
base = pd.merge(base, densidad_habitacion[['codusu', 'nro_hogar', 'densidad_habitacion']],
                on=['codusu', 'nro_hogar'], how='left')

print(base['densidad_habitacion'])
print("Cantidad de missing values en categoria_educacion: ", base['proporcion_ocupados_hogar'].isna().sum())

KeyError: 'densidad_habitacion'

In [38]:
#Cuarta: Índice de carga familiar (incluye menores de 6 años, discapacitados y adultos mayores-jubilados-)

# Contamos la cantidad personas menores de 6 años, discapacitados y jubilados que tiene cada hogar (numerador)
dependientes_hogar = base[(base['cat_inac'] == 1) | (base['cat_inac'] == 5) | (base['cat_inac'] == 6)].groupby(['codusu', 'nro_hogar'])['cat_inac'].count().reset_index()
dependientes_hogar.rename(columns={'cat_inac': 'dependientes_hogar'}, inplace=True)

# Contamos el total de miembros del hogar (denominador)
total_miembros_por_hogar = base.groupby(['codusu', 'nro_hogar'])['cat_inac'].count().reset_index()
total_miembros_por_hogar.rename(columns={'cat_inac': 'total_miembros_por_hogar'}, inplace=True)

# Unimos ambos dataframes
dependientes_hogar = pd.merge(total_miembros_por_hogar, dependientes_hogar, on=['codusu', 'nro_hogar'], how='left')
print(dependientes_hogar) #dependientes_hogar genera missing values cuando no hay personas dependientes en el hogar

# Reemplazamos los NaN con 0 en 'dependientes_hogar'
proporciones['dependientes_hogar'] = proporciones['dependientes_hogar'].fillna(0)

# Paso 5: Calcular la proporción
dependientes_hogar['prop_dependientes_hogar'] = dependientes_hogar['dependientes_hogar'] / dependientes_hogar['total_miembros_por_hogar']

# Paso 6: Unir las proporciones al DataFrame original
base = pd.merge(base, dependientes_hogar[['codusu', 'nro_hogar', 'prop_dependientes_hogar']], on=['codusu', 'nro_hogar'], how='left')

# Verificar los resultados
print(base['prop_dependientes_hogar'])
print("Cantidad de missing values en prop_dependientes_hogar: ", base['prop_dependientes_hogar'].isna().sum())

                             codusu  nro_hogar  total_miembros_por_hogar  \
0                          125463          1.0                         4   
1                          125571          1.0                         4   
2                          125591          1.0                         3   
3                          126052          1.0                         2   
4                          126339          1.0                         1   
...                             ...        ...                       ...   
1197  TQSMNOSTXHKOKNCDEHPJB00857737        1.0                         4   
1198  TQSMNOSVQHJOMMCDEHPJB00795313        1.0                         5   
1199  TQSMNOSXWHMMKQCDEHPJB00856905        1.0                         4   
1200  TQSMNOURSHMOKMCDEHPJB00859671        1.0                         2   
1201  TQTMNOSTXHKOKNCDEHPJB00858768        1.0                         2   

      dependientes_hogar  
0                    1.0  
1                    NaN  
2     

KeyError: 'dependientes_hogar'

In [ ]:
base[['proporcion_ocupados_hogar', 'categoria_educacion', 'densidad_habitacion', 'prop_dependientes_hogar', 'cat_inac']].describe()

In [50]:
created_var=['proporcion_ocupados_hogar', 'categoria_educacion', 'densidad_habitacion', 'prop_dependientes_hogar']
print(base[created_var].isnull().sum())

KeyError: "['densidad_habitacion', 'prop_dependientes_hogar'] not in index"

In [52]:
# Descripción básica de las variables de interés
base[created_var].describe() #no hace el describe de categoria_educacion porque es string

KeyError: "['densidad_habitacion', 'prop_dependientes_hogar'] not in index"

In [ ]:
#Definimos una función para calcular la proporción de desocupados por año
def proporcion_cate_educ(base, ano):
    total_desocupados = base[(base['categoria_educacion'] == alto) & (base['ano4'] == ano)]
    total_muestra = subset[(subset['ano4'] == ano)]
    
    proporcion = (total_desocupados['estado'].count() / total_muestra['estado'].count()) * 100
    return proporcion

# Mostrar proporciones de desocupados para cada año
print(f'Proporción de desocupados en 2004: {proporcion_desocupados(subset, 2004):.2f}%')
print(f'Proporción de desocupados en 2024: {proporcion_desocupados(subset, 2024):.2f}%')

#### Ejercicio 6

In [ ]:
#Tasa de desocupación con el PONDERA

import pandas as pd

# Asumamos que tu DataFrame se llama 'base' y tiene las siguientes columnas:
# 'codusu' para identificar el hogar,
# 'estado_desocupacion' (por ejemplo, 1 para desocupado y 0 para ocupado),
# 'ponderador' que permite expandir la muestra.

# Paso 1: Filtrar para el primer trimestre de 2024
# Suponiendo que hay una columna 'fecha' que indica el trimestre (deberías ajustarlo a tu caso).
base['fecha'] = pd.to_datetime(base['fecha'])  # Asegúrate de que la columna de fecha esté en el formato correcto
trimestre_2024 = base[(base['fecha'] >= '2024-01-01') & (base['fecha'] <= '2024-03-31')]

# Contar hogares con al menos un desocupado
hogares_desocupados = unicos_hogares[unicos_hogares['estado_desocupacion'] == 1]

# Paso 3: Calcular la tasa de hogares con desocupación
# Sumar los ponderadores para calcular la expansión
total_hogares = unicos_hogares['ponderador'].sum()
hogares_con_desocupacion = hogares_desocupados['ponderador'].sum()

# Calcular la tasa
tasa_hogares_desocupacion = hogares_con_desocupacion / total_hogares * 100  # En porcentaje

# Mostrar resultados
print(f"Tasa de hogares con desocupación en el primer trimestre de 2024: {tasa_hogares_desocupacion:.2f}%")


In [ ]:
import pandas as pd

# Supongamos que tu DataFrame se llama 'base' y tiene las siguientes columnas:
# 'cod_hogar', 'estado_desocupacion', 'ponderador'

# Paso 1: Colapsar a nivel de hogar
# Crear un nuevo DataFrame que indique si hay al menos un desocupado en el hogar
hogares_desocupados = base.groupby('cod_hogar').agg(
    desocupado=('estado_desocupacion', lambda x: 1 if (x == 1).any() else 0),  # 1 si hay al menos un desocupado
    ponderador_total=('ponderador', 'sum')  # Sumar el ponderador
).reset_index()

# Paso 2: Calcular la tasa de hogares con desocupación
total_hogares = hogares_desocupados['ponderador_total'].sum()  # Total de hogares ponderados
hogares_con_desocupacion = hogares_desocupados[hogares_desocupados['desocupado'] == 1]['ponderador_total'].sum()  # Total de hogares con desocupación

# Calcular la tasa
tasa_hogares_desocupacion = hogares_con_desocupacion / total_hogares * 100  # En porcentaje

# Mostrar resultados
print(f"Tasa de hogares con desocupación: {tasa_hogares_desocupacion:.2f}%")
